In [22]:
%pip install kagglehub

Note: you may need to restart the kernel to use updated packages.


In [23]:
%pip install streamlit

Note: you may need to restart the kernel to use updated packages.


In [24]:
import kagglehub

path = kagglehub.dataset_download("pranay13257/f1-dnf-classification")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\aryan\.cache\kagglehub\datasets\pranay13257\f1-dnf-classification\versions\1


In [25]:
%pip install torch torchvision torchaudio scikit-learn xgboost matplotlib numpy pandas xgboost

Note: you may need to restart the kernel to use updated packages.


In [26]:
import pandas as pd
import numpy as np

In [27]:
df = pd.read_csv(r'C:\Users\aryan\Downloads\Machine Learning\GDG\f1_dnf.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 31 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   resultId         10000 non-null  int64  
 1   raceId           10000 non-null  int64  
 2   year             10000 non-null  int64  
 3   round            10000 non-null  int64  
 4   grid             10000 non-null  int64  
 5   positionOrder    10000 non-null  int64  
 6   points           9029 non-null   float64
 7   laps             9022 non-null   float64
 8   milliseconds     8982 non-null   object 
 9   fastestLap       10000 non-null  object 
 10  rank             10000 non-null  object 
 11  fastestLapTime   10000 non-null  object 
 12  fastestLapSpeed  9047 non-null   object 
 13  driverRef        10000 non-null  object 
 14  surname          10000 non-null  object 
 15  forename         10000 non-null  object 
 16  dob              10000 non-null  object 
 17  nationality_x

In [28]:
df['nationality_y'].values

array(['British', 'British', 'Italian', ..., 'French', 'British',
       'Canadian'], shape=(10000,), dtype=object)

In [29]:
df.head()

,resultId,raceId,year,round,grid,positionOrder,points,laps,milliseconds,fastestLap,...,circuitRef,circuitId,name_y,location,country,lat,lng,alt,date,target_finish
0,2460,136,2002,13,11,4,3.0,77.0,NaN,\N,...,hungaroring,11,McLaren,Budapest,Hungary,47.5789,19.24860,264,2002-08-18,1
1,11565,483,1981,1,23,21,0.0,16.0,\N,\N,...,long_beach,43,McLaren,California,USA,33.7651,-118.18900,12,1981-03-15,0
2,18661,772,1958,8,0,26,0.0,0.0,\N,\N,...,nurburgring,20,Maserati,Nürburg,Germany,50.3356,6.94750,578,1958-08-03,0
3,25121,1058,2021,8,19,16,0.0,69.0,\N,57,...,red_bull_ring,70,Haas F1 Team,Spielberg,Austria,47.2197,14.76470,678,2021-06-27,0
4,8863,383,1988,12,0,30,0.0,0.0,\N,\N,...,monza,14,Euro Brun,Monza,Italy,45.6156,9.28111,162,1988-09-11,0


In [30]:
df['milliseconds'] = pd.to_numeric(df['milliseconds'], errors='coerce')
df['fastestLapSpeed'] = pd.to_numeric(df['fastestLapSpeed'], errors='coerce')
df['fastestLap'] = pd.to_numeric(df['fastestLap'], errors='coerce')
df['rank'] = pd.to_datetime(df['rank'], errors='coerce').fillna(0)
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['dob'] = pd.to_datetime(df['dob'], errors='coerce')

C:\Users\aryan\AppData\Local\Temp\ipykernel_19188\2433207429.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['rank'] = pd.to_datetime(df['rank'], errors='coerce').fillna(0)
C:\Users\aryan\AppData\Local\Temp\ipykernel_19188\2433207429.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['rank'] = pd.to_datetime(df['rank'], errors='coerce').fillna(0)


In [31]:
numeric_cols = ['points', 'laps', 'milliseconds', 'fastestLapSpeed']
for col in numeric_cols:
    df[col] = df[col].fillna(df[col].median())

In [32]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['race_month'] = df['date'].dt.month

In [33]:
def time_to_seconds(t):
    if isinstance(t, str) and ':' in t:
        m, s = t.split(':')
        try:
            return int(m)*60 + float(s)
        except:
            return None
    return None

df['fastestLapTime'] = df['fastestLapTime'].apply(time_to_seconds)
df['fastestLapTime'] = df['fastestLapTime'].fillna(df['fastestLapTime'].median())

In [34]:
cat_cols = ['driverRef','surname','forename','nationality_x','constructorRef',
            'name','nationality_y','circuitRef','name_y','location','country']

for col in cat_cols:
    df[col] = df[col].astype(str).str.strip().str.lower()

df[cat_cols].isnull().sum()

driverRef         0
surname           0
forename          0
nationality_x     0
constructorRef    0
name              0
nationality_y     0
circuitRef        0
name_y            0
location          0
country           0
dtype: int64

In [35]:
df = df.drop_duplicates()

In [36]:
df = df.drop(['resultId', 'raceId', 'surname', 'forename', 'dob', 'name_y', 'date'], axis=1)

In [37]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

encoding_cols = ['driverRef', 'nationality_x', 'country', 'location', 'circuitRef', 'name', 'constructorRef', 'nationality_y']
label_enc = LabelEncoder()

for col in encoding_cols:
    df[col] = label_enc.fit_transform(df[col])

X = df
y = df['target_finish']
X.drop(columns='target_finish', inplace=True)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [38]:
y.shape, X.shape

((10000,), (10000, 26))

In [39]:
from xgboost import XGBClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

model = XGBClassifier(
    n_estimators = 300,
    learning_rate = 0.05,
    max_depth = 7,
    criterion = 'log_loss',
    random_state = 42
)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("MAE: ", mean_absolute_error(y_test, y_pred))
print("RMSE: ", mean_squared_error(y_test, y_pred))
print("R^2: ", r2_score(y_test,y_pred))


c:\Users\aryan\anaconda3\envs\myenv\Lib\site-packages\xgboost\training.py:183: UserWarning: [09:05:28] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "criterion" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


MAE:  0.013
RMSE:  0.013
R^2:  0.9353745047450027


In [40]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(
    n_estimators=300,
    max_depth=15,
    criterion='log_loss',
    random_state=42
)

rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

print("MAE: ", mean_absolute_error(y_test, y_pred_rf))
print("RMSE: ", mean_squared_error(y_test, y_pred_rf))
print("R^2: ", r2_score(y_test,y_pred_rf))

MAE:  0.0125
RMSE:  0.0125
R^2:  0.9378601007163487


In [42]:
import joblib

joblib.dump(rf_model, "rf_model.pkl")
joblib.dump(label_enc, "label_encoder.pkl")


['label_encoder.pkl']